# Imports

In [30]:
import duckdb
import dtale
import pandas as pd
import requests
import re


In [2]:
from dotenv import dotenv_values, find_dotenv
from pathlib import Path

# Config

## Data Paths

In [3]:
path_data = Path() / '..' / 'data'
path_data_raw = path_data / 'raw'

## Credentials

In [4]:
config = dotenv_values(find_dotenv())
ad_api_key = config.get('AMBERDATA_API_KEY')

# Helper Functions

In [38]:
def camel_to_snake(name):
    return re.sub(r'(?<!^)(?=[A-Z])', '_', name).lower()

def convert_df_columns_to_snake_case(df):
    df.columns = [camel_to_snake(col) for col in df.columns]
    return df

def convert_df_columns_to_datetime(df, columns, unit='ms'):
    for column in columns:
        df[column] = pd.to_datetime(df[column], unit=unit)

    return df

# Get Data

In [5]:
headers = {
    "accept": "application/json",
    'Accept-Encoding': 'gzip, deflate, br',
    "x-api-key": ad_api_key
}

In [6]:
url = "https://api.amberdata.com/markets/futures/exchanges/reference"
params = {
    'exchange':'binance,bybit',
    'includeInactive':'true',
}

list_instruments = []
while url is not None:
    print(url)
    try:
        response = requests.get(url, headers=headers, params=params)
        payload = response.json().get('payload', {})
        metadata = payload.get('metadata', {})
        data = payload.get('data', {})
        chunk_instruments = pd.DataFrame(data)
        list_instruments.append(chunk_instruments)
        url = metadata.get('next', None)
    except Exception as e:
        print(e)

https://api.amberdata.com/markets/futures/exchanges/reference
https://api.amberdata.com/markets/futures/exchanges/reference?cursor=N4IgpgHgxgFghgOwOZhALhAIwJYMVMAGkwE8cAXEQkXKAGwFcATMASTynOwDdU1yATgyI0E9ZmADyA7Elxw6AJTAAzMALBi%2BKhQGcRtRiwCqCXQwAOFgPYDyYJgFFo8ZGF3oddfdS4BbMAAxWz84Sgw-bDo6bH0oawQmD2oVELD0EAArXQSqEAs4FHQARmpdbAAvPgBWAAZagF8gA
https://api.amberdata.com/markets/futures/exchanges/reference?cursor=N4IgpgHgxgFghgOwOZhALhAIwJYMVMAGkwE8cAXEQkXKAGwFcATMASTynOwDdU1yATgyI0E9ZmADyA7Elxw6AJTAAzMALBi%2BKhQGcRtRiwCqCXQwAOFgPYDyYJgFFo8ZGF3oddfdS4BbMAAxWz84Sgw-bDo6bH0oawQmD2oVELD0EAArXQSqEAs4FHQAJmpdbAAvPgBWAAZagF8gA
https://api.amberdata.com/markets/futures/exchanges/reference?cursor=N4IgpgHgxgFghgOwOZhALhAIwJYMVMAGkwE8cAXEQkXKAGwFcATMASTynOwDdU1yATgyI0E9ZmADyA7Elxw6AJTAAzMALBi%2BKhQGcRtRiwCqCXQwAOFgPYDyYJgFFo8ZGF3oddfdS4BbMAAxWz84Sgw-bDo6bH0oawQmD2oVELD0EAArXQSqEAs4FHQAZmpdbAAvPgBWAAZagF8gA


In [39]:
datetime_cols = ['listing_timestamp', 'contract_expiration_timestamp']

df_instruments = (
    pd.concat(list_instruments, axis=0)
    .pipe(convert_df_columns_to_snake_case)
    .pipe(convert_df_columns_to_datetime, columns=datetime_cols)
)

/tmp/ipykernel_1240048/4265807693.py:4: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



# Ingest Data

In [42]:
conn = duckdb.connect(path_data / "crypto_data.db")

In [50]:
conn.execute("DROP TABLE IF EXISTS amberdata.exchange_reference")
conn.register("temp_df", df_instruments)
conn.execute("""
    CREATE TABLE amberdata.exchange_reference AS
    SELECT * FROM temp_df
    ORDER BY exchange, instrument
""")

In [55]:
result = conn.execute("""
    SELECT 
        exchange,
        instrument,
    FROM amberdata.exchange_reference
    WHERE 
        contract_period = 'perpetual'
        AND quote_symbol = 'USDT'
        AND exchange_enabled = True
""").fetchdf()

In [59]:
conn.close()

2025-01-14 13:28:04,695 - INFO     - Executing shutdown due to inactivity...
2025-01-14 13:28:04,703 - INFO     - Executing shutdown...
2025-01-14 13:28:04,704 - INFO     - Not running with the Werkzeug Server, exiting by searching gc for BaseWSGIServer
